# Preprocess the data
* ### Feature Selection?
* ### Split Data into Train and Test
* ### Standardize and Normalize the magnitude of numeric features using a scaler


### Import Packages

In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib
import plotly.figure_factory as ff
from IPython.core.display import display
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# pd.set_option('display.max_columns', None)
# pd.reset_option('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

In [159]:
# define useful function to create scatterplots of target feature against desired columns
def scatterplots(df, target, columns, ncol=None, figsize=(20, 25)):
    if ncol is None:
        ncol = len(columns)
    nrow = int(np.ceil(len(columns) / ncol))
    fig, axes = plt.subplots(nrow, ncol, figsize=figsize, squeeze=False)
    fig.subplots_adjust(wspace=0.7, hspace=0.7)
    for i, col in enumerate(columns):
        # ax = axes.flatten()[i]
        # ax.scatter(x = col, y = target, data=df, alpha=0.5)
        sns.regplot(ax=axes.flatten()[i], x=col, y=target, data=df, scatter_kws={'alpha':0.5})
        # ax.set(xlabel=col, ylabel=target)
    nsubplots = nrow * ncol
    for empty in range(i+1, nsubplots):
        axes.flatten()[empty].set_visible(False)

### Load data

In [160]:
pm10_2017_2019 = r'../../data/processed/pm10_2017_2019'
pm10_year_diff = r'../../data/processed/pm10_year_diff'
pm10_year_pct_change = r'../../data/processed/pm10_year_pct_change'
pm10_year_diff_top_corr =  r'../../data/processed/pm10_year_diff_top_corr'
pm10_year_pct_change_top_corr = r'../../data/processed/pm10_year_pct_change_top_corr'

df_norm = pd.read_csv(pm10_2017_2019)
df_year_diff = pd.read_csv(pm10_year_diff)
df_year_diff_top = pd.read_csv(pm10_year_diff_top_corr)
df_year_pct_change = pd.read_csv(pm10_year_pct_change)
df_year_pct_change_top = pd.read_csv(pm10_year_pct_change_top_corr)

In [161]:
# concatenate all features
# after concentate look at features and see if need to standardize or normalize

display(df_year_pct_change)
display(df_year_diff)

,AQI_2019,Civilian_labor_force_2017_2018_pct_change,Employed_2017_2018_pct_change,Unemployed_2017_2018_pct_change,Unemployment_rate_2017_2018_pct_change,"Poverty Estimate, All Ages_2017_2018_pct_change",90% CI LB All Ages_2017_2018_pct_change,90% CI UB All Ages_2017_2018_pct_change,"Poverty Percent, All Ages_2017_2018_pct_change",90% CI LB percent_2017_2018_pct_change,...,HWAC_MALE_ratio_2018_2019_pct_change,HWAC_FEMALE_ratio_2018_2019_pct_change,HBAC_MALE_ratio_2018_2019_pct_change,HBAC_FEMALE_ratio_2018_2019_pct_change,HIAC_MALE_ratio_2018_2019_pct_change,HIAC_FEMALE_ratio_2018_2019_pct_change,HAAC_MALE_ratio_2018_2019_pct_change,HAAC_FEMALE_ratio_2018_2019_pct_change,HNAC_MALE_ratio_2018_2019_pct_change,HNAC_FEMALE_ratio_2018_2019_pct_change
0,20.073389,0.013844,0.020189,-0.128942,-0.139535,-0.038496,-0.045462,-0.032514,-0.035928,-0.038961,...,0.026476,0.031723,0.032864,0.040322,0.054139,0.074790,0.122425,-0.041309,0.055424,0.023801
1,12.958515,0.028336,0.033239,-0.093125,-0.128205,-0.051402,-0.053703,-0.049561,-0.070313,-0.078947,...,0.022018,0.030026,0.037618,0.016357,-0.005556,0.051458,-0.087266,0.243875,0.072299,-0.088200
2,13.125000,-0.001903,0.002235,-0.093261,-0.093023,-0.059333,-0.062900,-0.056488,-0.056338,-0.058201,...,0.025343,0.032246,-0.001476,0.063161,0.004428,-0.048117,-0.093386,0.020489,-0.010560,0.033049
3,11.247911,0.006385,0.011546,-0.038812,-0.039216,0.128447,0.155791,0.105858,0.126888,0.157191,...,0.042465,0.013679,0.095179,-0.115560,0.036155,-0.021558,-0.158081,0.130874,0.496745,0.122559
4,17.413408,0.003142,0.003413,-0.001463,0.000000,-0.049222,-0.038434,-0.056579,-0.062112,-0.053435,...,0.008941,0.001484,-0.001274,-0.048402,0.019675,-0.012253,0.074008,-0.012205,0.018867,-0.083067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,10.503937,-0.009622,-0.006847,-0.102491,-0.103448,0.040280,0.012364,0.060632,0.034884,0.000000,...,0.008593,0.038840,0.049928,0.096699,0.025571,-0.000068,0.352849,0.047547,0.142779,-0.166724
204,14.431990,-0.014080,-0.009978,-0.111942,-0.100000,-0.003911,-0.005839,-0.002167,0.000000,0.000000,...,0.015727,0.015638,0.013946,0.019648,0.007658,0.006921,-0.001563,0.007946,0.016960,0.001314
205,15.656425,-0.002127,0.000323,-0.084676,-0.103448,0.044369,0.040462,0.047385,0.041667,0.023810,...,0.020747,0.021930,0.045184,0.029480,0.100848,0.057137,0.117484,0.185540,0.058094,0.251201
206,13.354142,-0.000829,-0.001614,0.019909,0.027778,-0.048956,-0.071456,-0.032807,-0.061856,-0.086420,...,0.006963,0.020251,-0.020994,0.047957,0.007984,-0.018559,-0.020887,0.018051,0.196858,0.397275


,AQI_2019,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018_2019_diff,HWAC_FEMALE_ratio_2018_2019_diff,HBAC_MALE_ratio_2018_2019_diff,HBAC_FEMALE_ratio_2018_2019_diff,HIAC_MALE_ratio_2018_2019_diff,HIAC_FEMALE_ratio_2018_2019_diff,HAAC_MALE_ratio_2018_2019_diff,HAAC_FEMALE_ratio_2018_2019_diff,HNAC_MALE_ratio_2018_2019_diff,HNAC_FEMALE_ratio_2018_2019_diff
0,20.073389,4312,6021,-1709,-0.6,-4148.0,-4526.0,-3770.0,-0.6,-0.6,...,0.049821,0.052091,0.005691,0.007374,0.003112,0.003709,0.002302,-0.000589,0.000933,0.000321
1,12.958515,5003,5641,-638,-0.5,-2312.0,-2147.0,-2477.0,-0.9,-0.9,...,0.050694,0.063381,0.009625,0.004769,-0.000580,0.004921,-0.002948,0.009569,0.001320,-0.001634
2,13.125000,-202,227,-429,-0.4,-2748.0,-2585.0,-2911.0,-1.2,-1.1,...,0.038534,0.043819,-0.000419,0.018125,0.000450,-0.005290,-0.004412,0.000886,-0.000438,0.001328
3,11.247911,130,211,-81,-0.4,3009.0,3302.0,2716.0,4.2,4.7,...,0.100582,0.027614,0.005440,-0.007088,0.036745,-0.023202,-0.007052,0.005474,0.004155,0.001367
4,17.413408,155,159,-4,0.0,-946.0,-599.0,-1293.0,-1.0,-0.7,...,0.148808,0.025037,-0.000588,-0.022055,0.013852,-0.008442,0.015924,-0.003012,0.002000,-0.008345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,10.503937,-466,-322,-144,-0.3,282.0,73.0,491.0,0.3,0.0,...,0.022589,0.079551,0.004548,0.006826,0.002271,-0.000005,0.006829,0.001137,0.001138,-0.002278
204,14.431990,-6690,-4550,-2140,-0.4,-696.0,-987.0,-405.0,0.0,0.0,...,0.110881,0.105975,0.008989,0.013032,0.002425,0.002219,-0.000115,0.000633,0.000344,0.000025
205,15.656425,-481,71,-552,-0.3,847.0,673.0,1021.0,0.2,0.1,...,0.047631,0.049311,0.003626,0.002820,0.007842,0.004613,0.003824,0.005808,0.000693,0.002434
206,13.354142,-40,-75,35,0.1,-455.0,-555.0,-355.0,-0.6,-0.7,...,0.049293,0.133734,-0.005776,0.012418,0.003351,-0.007659,-0.002894,0.002228,0.008761,0.012859


In [162]:
df = pd.merge(df_year_diff, df_year_pct_change)
display(df)


,AQI_2019,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018_2019_pct_change,HWAC_FEMALE_ratio_2018_2019_pct_change,HBAC_MALE_ratio_2018_2019_pct_change,HBAC_FEMALE_ratio_2018_2019_pct_change,HIAC_MALE_ratio_2018_2019_pct_change,HIAC_FEMALE_ratio_2018_2019_pct_change,HAAC_MALE_ratio_2018_2019_pct_change,HAAC_FEMALE_ratio_2018_2019_pct_change,HNAC_MALE_ratio_2018_2019_pct_change,HNAC_FEMALE_ratio_2018_2019_pct_change
0,20.073389,4312,6021,-1709,-0.6,-4148.0,-4526.0,-3770.0,-0.6,-0.6,...,0.026476,0.031723,0.032864,0.040322,0.054139,0.074790,0.122425,-0.041309,0.055424,0.023801
1,12.958515,5003,5641,-638,-0.5,-2312.0,-2147.0,-2477.0,-0.9,-0.9,...,0.022018,0.030026,0.037618,0.016357,-0.005556,0.051458,-0.087266,0.243875,0.072299,-0.088200
2,13.125000,-202,227,-429,-0.4,-2748.0,-2585.0,-2911.0,-1.2,-1.1,...,0.025343,0.032246,-0.001476,0.063161,0.004428,-0.048117,-0.093386,0.020489,-0.010560,0.033049
3,11.247911,130,211,-81,-0.4,3009.0,3302.0,2716.0,4.2,4.7,...,0.042465,0.013679,0.095179,-0.115560,0.036155,-0.021558,-0.158081,0.130874,0.496745,0.122559
4,17.413408,155,159,-4,0.0,-946.0,-599.0,-1293.0,-1.0,-0.7,...,0.008941,0.001484,-0.001274,-0.048402,0.019675,-0.012253,0.074008,-0.012205,0.018867,-0.083067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,10.503937,-466,-322,-144,-0.3,282.0,73.0,491.0,0.3,0.0,...,0.008593,0.038840,0.049928,0.096699,0.025571,-0.000068,0.352849,0.047547,0.142779,-0.166724
204,14.431990,-6690,-4550,-2140,-0.4,-696.0,-987.0,-405.0,0.0,0.0,...,0.015727,0.015638,0.013946,0.019648,0.007658,0.006921,-0.001563,0.007946,0.016960,0.001314
205,15.656425,-481,71,-552,-0.3,847.0,673.0,1021.0,0.2,0.1,...,0.020747,0.021930,0.045184,0.029480,0.100848,0.057137,0.117484,0.185540,0.058094,0.251201
206,13.354142,-40,-75,35,0.1,-455.0,-555.0,-355.0,-0.6,-0.7,...,0.006963,0.020251,-0.020994,0.047957,0.007984,-0.018559,-0.020887,0.018051,0.196858,0.397275


Decide to only have a df retaining features of difference and percent changes
because these column are stationarized random variables and behave similar to independent random variables
whihc will give us more suitable results for modeling

# perform eda to decide whether we need to standardize or normalize
# then split data before standardize and normalize

df.describe()

Can tell we need to standardize already by seeing some columns with varying means
especially between Civilian_labor_force_2017_2018_diff with mean 2500 and Unemployment_rate_2017_2018_diff	-.4

In [163]:
# Run Shapiro test on all features and count number of features with a p score less than .05 \
# Features with a shapiro p score less than .05 are considered not gaussian

count = 0
for feature in df.columns:
    shapiro_test = stats.shapiro(df[feature])
    if shapiro_test.pvalue < 0.05:
        count = count + 1
print(" {} features were deemed to not have a normal distribution out of {} features".format(count, len(df)))

 1046 features were deemed to not have a normal distribution out of 208 features


In [164]:
# sns.distplot(x)
shapiro_test = stats.shapiro(df['AQI_2019'])
shapiro_test.pvalue

2.866333004281074e-10

# Result
* Standardization of data should be performed because features have discrepancy in magnitudes from looking at means of some features
* Normalization of data should be performed because majority of the features are reported wht non gaussian distributions

# split data then scale

In [165]:
X = df.drop('AQI_2019', axis=1)
y = df['AQI_2019']

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale Data

In [167]:
# checking whether we have negative observations

for cols in df.columns.tolist()[1:]:
     data = df.loc[df[cols] > 0]
data

,AQI_2019,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018_2019_pct_change,HWAC_FEMALE_ratio_2018_2019_pct_change,HBAC_MALE_ratio_2018_2019_pct_change,HBAC_FEMALE_ratio_2018_2019_pct_change,HIAC_MALE_ratio_2018_2019_pct_change,HIAC_FEMALE_ratio_2018_2019_pct_change,HAAC_MALE_ratio_2018_2019_pct_change,HAAC_FEMALE_ratio_2018_2019_pct_change,HNAC_MALE_ratio_2018_2019_pct_change,HNAC_FEMALE_ratio_2018_2019_pct_change
0,20.073389,4312,6021,-1709,-0.6,-4148.0,-4526.0,-3770.0,-0.6,-0.6,...,0.026476,0.031723,0.032864,0.040322,0.054139,0.074790,0.122425,-0.041309,0.055424,0.023801
2,13.125000,-202,227,-429,-0.4,-2748.0,-2585.0,-2911.0,-1.2,-1.1,...,0.025343,0.032246,-0.001476,0.063161,0.004428,-0.048117,-0.093386,0.020489,-0.010560,0.033049
3,11.247911,130,211,-81,-0.4,3009.0,3302.0,2716.0,4.2,4.7,...,0.042465,0.013679,0.095179,-0.115560,0.036155,-0.021558,-0.158081,0.130874,0.496745,0.122559
5,23.346218,75798,74930,868,-0.1,-35219.0,-34873.0,-35565.0,-1.2,-1.2,...,0.005234,0.006132,0.022117,0.014747,0.005855,0.009076,0.018505,0.022720,0.007375,0.011679
6,14.785714,2589,2632,-43,-0.2,-480.0,-996.0,36.0,-0.5,-0.7,...,0.014403,0.014350,0.022827,-0.022980,0.009790,-0.003506,0.058975,0.070805,-0.158172,0.005725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,14.556364,2142,2239,-97,-0.2,-3990.0,-4291.0,-3689.0,-2.2,-2.3,...,0.018764,0.025343,0.029254,0.036552,0.001308,-0.004244,-0.020316,0.050468,0.073669,0.117225
201,14.106628,3574,3852,-278,-0.4,-3765.0,-3117.0,-4413.0,-1.6,-1.3,...,0.007466,0.009925,0.013563,0.020211,0.022123,0.013935,0.039837,0.035663,0.073742,0.014956
204,14.431990,-6690,-4550,-2140,-0.4,-696.0,-987.0,-405.0,0.0,0.0,...,0.015727,0.015638,0.013946,0.019648,0.007658,0.006921,-0.001563,0.007946,0.016960,0.001314
205,15.656425,-481,71,-552,-0.3,847.0,673.0,1021.0,0.2,0.1,...,0.020747,0.021930,0.045184,0.029480,0.100848,0.057137,0.117484,0.185540,0.058094,0.251201


In [168]:
# because we negative observation we cannot perform logarithmic scale and will instead perform standardization
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[ 0.2327869 ,  0.16039692,  0.18413171, ...,  0.41338241,
        -0.05358407,  0.34944449],
       [-0.08904047,  0.2148679 , -0.80800341, ..., -0.11767301,
        -0.17803333, -0.28456306],
       [-0.25122218, -0.38406301,  0.3646109 , ...,  0.74260694,
         0.88470847, -0.59002294],
       ...,
       [-0.37625047, -0.53810594,  0.44701082, ...,  0.57427211,
        -0.8572769 ,  0.63239902],
       [-0.26859427, -0.45315121,  0.50338972, ...,  3.79653544,
         1.55968692,  2.98472863],
       [ 0.21034268,  0.10105353,  0.28354539, ...,  0.22859522,
         0.19255954, -0.05655795]])

In [169]:
# scale test data using the scaler fitted from the training set

X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-0.04909735,  0.08943472, -0.36798114, ..., -0.22283007,
         0.01438031, -0.05467762],
       [-0.15561232, -0.27711999,  0.3305834 , ...,  0.29786259,
         0.134467  ,  0.06603427],
       [ 0.09875557,  0.40214312, -0.81400826, ..., -0.88521186,
        -1.05883252, -1.12358167],
       ...,
       [-0.31677961, -0.43553559,  0.32958259, ..., -0.18350479,
        -0.12925027, -0.25688545],
       [ 0.96304845,  1.14674643, -0.52844415, ...,  0.30202937,
        -0.14120865, -0.14762325],
       [-0.10793419, -0.26974892,  0.43633553, ..., -0.62611886,
         0.05270872,  0.62190022]])

In [170]:
# retain X_train_scale as a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
display(X_train_scaled)

# retain X_test_scale as a dataframe
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)
display(X_test_scaled)


,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,90% CI UB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018_2019_pct_change,HWAC_FEMALE_ratio_2018_2019_pct_change,HBAC_MALE_ratio_2018_2019_pct_change,HBAC_FEMALE_ratio_2018_2019_pct_change,HIAC_MALE_ratio_2018_2019_pct_change,HIAC_FEMALE_ratio_2018_2019_pct_change,HAAC_MALE_ratio_2018_2019_pct_change,HAAC_FEMALE_ratio_2018_2019_pct_change,HNAC_MALE_ratio_2018_2019_pct_change,HNAC_FEMALE_ratio_2018_2019_pct_change
29,0.232787,0.160397,0.184132,-0.708151,-0.032929,-0.052244,-0.012659,-0.358545,-0.408037,-0.299123,...,-0.485544,-0.253994,0.714708,0.228908,-0.334144,0.326982,0.233400,0.413382,-0.053584,0.349444
19,-0.089040,0.214868,-0.808003,-1.896018,-0.366625,-0.255945,-0.478881,-0.358545,-0.272170,-0.449893,...,-0.301190,-0.285916,-0.235271,-0.246884,-0.125805,0.003241,0.110718,-0.117673,-0.178033,-0.284563
55,-0.251222,-0.384063,0.364611,-0.114218,0.171585,0.192251,0.148936,0.144708,0.203365,0.077801,...,2.292255,2.838236,0.982626,1.275438,1.338019,1.346011,3.147608,0.742607,0.884708,-0.590023
93,-0.363063,-0.509996,0.406645,0.182749,0.164637,0.192520,0.134550,0.144708,0.271299,0.002416,...,-0.055012,0.134957,-0.088771,-0.559957,-0.978862,-0.728412,-0.180069,-0.680062,0.142632,0.170238
181,0.161016,0.018098,0.375286,0.776682,-0.053591,-0.097600,-0.007554,-0.358545,-0.408037,-0.299123,...,0.348702,0.965426,0.601616,0.029213,-0.239003,1.466002,0.684790,0.469880,-0.706612,-0.041992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.973573,0.912621,0.124417,0.479716,-0.753437,-0.800489,-0.699322,-0.861798,-0.883572,-0.826818,...,0.157717,0.073935,0.290348,0.266264,0.033105,1.119985,-0.034476,0.292918,0.164696,0.704315
14,-0.026907,0.191880,-0.583155,-0.411185,-1.187242,-1.037827,-1.333915,-0.789905,-0.679772,-0.902203,...,-0.555474,-0.474261,-0.321066,-0.358884,-0.165868,-0.051595,0.033192,-0.209806,-0.098813,-0.067015
92,-0.376250,-0.538106,0.447011,0.479716,0.511314,0.493196,0.526518,1.726361,1.562038,1.887039,...,0.339371,0.334314,-0.001595,-0.278764,-0.470995,0.970671,0.118684,0.574272,-0.857277,0.632399
179,-0.268594,-0.453151,0.503390,0.776682,0.239330,0.237250,0.239805,0.432282,0.407166,0.454726,...,0.702801,0.970580,0.932521,-3.081165,-1.572072,-1.606897,-0.010341,3.796535,1.559687,2.984729


,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,90% CI UB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018_2019_pct_change,HWAC_FEMALE_ratio_2018_2019_pct_change,HBAC_MALE_ratio_2018_2019_pct_change,HBAC_FEMALE_ratio_2018_2019_pct_change,HIAC_MALE_ratio_2018_2019_pct_change,HIAC_FEMALE_ratio_2018_2019_pct_change,HAAC_MALE_ratio_2018_2019_pct_change,HAAC_FEMALE_ratio_2018_2019_pct_change,HNAC_MALE_ratio_2018_2019_pct_change,HNAC_FEMALE_ratio_2018_2019_pct_change
161,-0.049097,0.089435,-0.367981,-1.005118,-0.012268,-0.026032,0.002099,-0.070972,-0.136303,0.002416,...,-0.081817,-0.222311,0.078516,0.371824,0.004697,0.660013,0.383413,-0.222830,0.014380,-0.054678
15,-0.155612,-0.277120,0.330583,-0.708151,0.155220,0.167292,0.141604,0.000922,-0.000435,0.002416,...,-0.398694,-0.350170,-0.018022,-0.407323,0.388912,0.738759,0.664864,0.297863,0.134467,0.066034
73,0.098756,0.402143,-0.814008,-1.005118,-0.896515,-0.687143,-1.107440,-0.789905,-0.611838,-0.977588,...,-0.946231,-0.886741,-0.809538,-0.114065,-0.990476,-1.171774,-0.395261,-0.885212,-1.058833,-1.123582
96,-0.380181,-0.551224,0.471697,-0.114218,0.145986,0.148416,0.142439,-0.214758,-0.272170,-0.148354,...,-0.648097,2.237407,0.785174,3.292703,1.988278,4.294620,-1.404437,2.040284,-1.229761,-2.981641
166,-0.437497,-0.518741,0.234172,-1.599052,0.165825,0.159956,0.170748,0.216602,0.135432,0.303956,...,3.984226,3.201610,-1.177042,2.262078,-0.373182,4.133276,6.577092,3.537599,-0.198774,1.552782
9,0.734548,0.493220,0.615480,1.073649,0.342821,0.230988,0.456440,0.072815,-0.136303,0.303956,...,-0.429145,-0.444403,-0.252843,-0.394805,-0.787781,-0.748656,-0.000998,-0.135109,-0.025371,0.038102
100,-0.381323,-0.557096,0.484374,0.776682,0.045695,0.016730,0.075425,-0.718011,-0.883572,-0.525278,...,1.465248,0.860484,2.428223,5.008282,7.003266,0.629081,2.693847,-2.761665,-0.919343,1.102956
135,-0.633535,-0.372569,-0.671893,0.182749,-1.402544,-1.288405,-1.511103,-0.358545,-0.272170,-0.449893,...,-1.289173,-1.613617,-0.820221,-0.687980,-0.619496,-0.498468,-0.289512,-0.315771,-0.329996,-0.368975
18,-0.424943,-0.535482,0.311902,-0.411185,0.283487,0.207012,0.360838,0.647961,0.203365,1.133190,...,-0.732137,-0.924439,0.107108,-0.282485,0.258725,0.041638,0.220111,-0.434099,-0.056668,-0.376632
148,-0.373714,-0.513369,0.387630,-1.302085,0.174327,0.198245,0.148287,0.144708,0.271299,0.002416,...,0.639782,-2.943164,-0.572126,-1.519750,-1.867157,-1.856327,5.495358,2.880903,-0.980191,-1.109994
